In [1]:
import numpy as np
import pandas as pd
# import matplotlib.pyplot as plt
# import seaborn as sns
# import scipy.stats as st
from statsmodels.formula.api import ols

# Task 2

Нужно предположить, какое `Revenue7` мы получаем по каждой стране на каждом источнике.

Опишите логику решения.

Возможно несколько вариантов решения.


## Installs

Organic: 11851

Facebook: 3137

Google: 3841

In [2]:
total_installs = pd.Series({
    "Organic": 11851,
    "Facebook": 3137,
    "Google": 3841
}).sort_index()

revenue = pd.read_excel(
    io="./DA_task_2.xlsx",
    usecols="A:F"
)
revenue.head()

,type,country,Spend,Clicks,Installs,Revenue7
0,Organic+FB+Google,US,NaN,NaN,5063.0,11345.630884
1,Organic+FB+Google,UK,NaN,NaN,2077.0,934.957527
2,Organic+FB+Google,BR,NaN,NaN,1826.0,174.960320
3,Organic+FB+Google,DE,NaN,NaN,1784.0,3301.434703
4,Organic+FB+Google,FR,NaN,NaN,1576.0,1013.666092


In [3]:
revenue.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35 entries, 0 to 34
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   type      35 non-null     object 
 1   country   35 non-null     object 
 2   Spend     15 non-null     float64
 3   Clicks    15 non-null     float64
 4   Installs  20 non-null     float64
 5   Revenue7  20 non-null     float64
dtypes: float64(4), object(2)
memory usage: 1.8+ KB


In [4]:
revenue.describe()

,Spend,Clicks,Installs,Revenue7
count,15.000000,15.000000,20.000000,20.000000
mean,4745.029667,8849.866667,941.450000,1105.512252
std,6462.067786,10770.983546,1192.484469,2521.484536
min,2.000000,0.000000,1.000000,0.000000
25%,1609.958681,747.500000,43.250000,88.029767
50%,2151.368265,5099.000000,578.000000,408.552635
75%,2893.170212,13765.000000,1499.500000,853.333572
max,17763.000442,36982.000000,5063.000000,11345.630884


Данные можно (и нужно) разделить на две группы:
1. Те, в которых известны суммарные `Installs` и `Revenue7` (назовём их `totals`)
2. Те, в которых известно разбиение `Spend` и `Clicks` по `type` и `country` (назовём их `by_type`).

In [5]:
totals = (revenue
    .loc[revenue["Clicks"].isna(), ["type", "country", "Installs", "Revenue7"]]
    .astype({"Installs": "int64"})
)

totals.head()

,type,country,Installs,Revenue7
0,Organic+FB+Google,US,5063,11345.630884
1,Organic+FB+Google,UK,2077,934.957527
2,Organic+FB+Google,BR,1826,174.960320
3,Organic+FB+Google,DE,1784,3301.434703
4,Organic+FB+Google,FR,1576,1013.666092


In [6]:
by_type = (revenue
    .loc[revenue["Clicks"].notna(), ["type", "country", "Spend", "Clicks"]]
    .astype({"Clicks": "int64"})
)

by_type.head()

,type,country,Spend,Clicks
20,FB,US,16009.235678,22297
21,FB,FR,2622.658306,13124
22,FB,IT,2343.566877,20595
23,FB,UK,2151.368265,6014
24,FB,BR,1865.899963,36982


In [7]:
totals.loc[:, ["type", "country"]].nunique()

type        1
country    20
dtype: int64

In [8]:
by_type.loc[:, ["type", "country"]].nunique()

type        2
country    12
dtype: int64

In [9]:
types = (pd
    .DataFrame(totals["type"].unique()[0].split("+"))
    .rename(
        mapper={0:"type"},
        axis=1
    )
)

countries = (pd
    .DataFrame(totals["country"].unique())
    .rename(
        mapper={0: "country"},
        axis=1
    )
)

print("""{}\n{}""".format(tuple(types["type"]), tuple(countries["country"])))

('Organic', 'FB', 'Google')
('US', 'UK', 'BR', 'DE', 'FR', 'IT', 'JP', 'ES', 'CA', 'KR', 'AU', 'NL', 'TW', 'SE', 'IR', 'PS', 'BN', 'GP', 'FJ', 'TL')


In [10]:
all_by_type = (types
    .merge(
        right=countries,
        how="cross"
    ) #Все возможные пары type-country.
    .merge(
        right=by_type,
        on=["type", "country"],
        how="left"
    ) #Данные, которые у нас есть.
)

all_by_type.loc[:, ["type", "country"]].nunique()

type        3
country    20
dtype: int64

In [11]:
# За Organic мы платим $0.0, а если положим, что данные полные,
# то за неуказанные пары type-country мы тоже платим $0.0.

all_by_type.loc[:, "Spend"] = all_by_type.loc[:, "Spend"].fillna(0)
all_by_type.head()

,type,country,Spend,Clicks
0,Organic,US,0.0,NaN
1,Organic,UK,0.0,NaN
2,Organic,BR,0.0,NaN
3,Organic,DE,0.0,NaN
4,Organic,FR,0.0,NaN


In [12]:
by_type_country=(all_by_type
    .groupby(["type", "country"])
    [["Spend", "Clicks"]]
    .sum()
    .reset_index()
)
by_type_country.columns = by_type_country.columns.to_series().apply(str.lower)

by_type_country.head()

,type,country,spend,clicks
0,FB,AU,0.000000,0.0
1,FB,BN,0.000000,0.0
2,FB,BR,1865.899963,36982.0
3,FB,CA,0.000000,0.0
4,FB,DE,1620.480017,4917.0


In [13]:
# Объединим оба датасета:
totals = (totals
    .merge(
        right=by_type_country.loc[by_type_country["type"] == "Google", :],
        on="country",
        suffixes=["", "_g"]
    )
    .merge(
        right=by_type_country.loc[by_type_country["type"] == "FB", :],
        on="country",
        suffixes=["", "_f"]
    )
)
totals.head()

,type,country,Installs,Revenue7,type_g,spend,clicks,type_f,spend_f,clicks_f
0,Organic+FB+Google,US,5063,11345.630884,Google,17443.027248,6435.0,FB,16009.235678,22297.0
1,Organic+FB+Google,UK,2077,934.957527,Google,0.000000,0.0,FB,2151.368265,6014.0
2,Organic+FB+Google,BR,1826,174.960320,Google,0.000000,0.0,FB,1865.899963,36982.0
3,Organic+FB+Google,DE,1784,3301.434703,Google,0.000000,0.0,FB,1620.480017,4917.0
4,Organic+FB+Google,FR,1576,1013.666092,Google,3076.126498,1151.0,FB,2622.658306,13124.0


In [14]:
totals = (totals
    .rename(
        mapper={
            "spend": "spend_g",
            "clicks": "clicks_g",
            "Installs": "installs",
            "Revenue7": "revenue7"
        },
        axis=1
    )
)

totals["spend_o"] = 0.0 #Хорошо, что не платим за Organic.
totals["clicks_o"] = np.nan #Плохо, что не знаем, сколько кликов пришло оттуда. 

totals = (totals
    .loc[
        :,
        [
            "country",
            "spend_g", "spend_f", "spend_o",
            "clicks_g", "clicks_f", "clicks_o",
            "installs", "revenue7"
        ]
    ].astype({
        "clicks_g": "int64",
        "clicks_f": "int64",
    })
)
totals

,country,spend_g,spend_f,spend_o,clicks_g,clicks_f,clicks_o,installs,revenue7
0,US,17443.027248,16009.235678,0.0,6435,22297,NaN,5063,11345.630884
1,UK,0.000000,2151.368265,0.0,0,6014,NaN,2077,934.957527
2,BR,0.000000,1865.899963,0.0,0,36982,NaN,1826,174.960320
3,DE,0.000000,1620.480017,0.0,0,4917,NaN,1784,3301.434703
4,FR,3076.126498,2622.658306,0.0,1151,13124,NaN,1576,1013.666092
5,IT,1947.870521,2343.566877,0.0,850,20595,NaN,1474,678.665733
6,JP,17763.000442,0.000000,0.0,5099,0,NaN,1080,1177.084200
7,ES,0.000000,1599.437345,0.0,0,14406,NaN,894,439.404414
8,CA,0.000000,0.000000,0.0,0,0,NaN,801,826.125587
9,KR,0.000000,0.000000,0.0,0,0,NaN,602,377.700856


Теперь есть красивый широкий датасет, на котором можно попытаться рассчитать конверсию.

In [15]:
totals.iloc[:, 1:].corr()

,spend_g,spend_f,spend_o,clicks_g,clicks_f,clicks_o,installs,revenue7
spend_g,1.000000,0.634803,NaN,0.990243,0.203855,NaN,0.621872,0.686372
spend_f,0.634803,1.000000,NaN,0.731347,0.549994,NaN,0.918144,0.953494
spend_o,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
clicks_g,0.990243,0.731347,NaN,1.000000,0.264338,NaN,0.696896,0.767946
clicks_f,0.203855,0.549994,NaN,0.264338,1.000000,NaN,0.658087,0.371575
clicks_o,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
installs,0.621872,0.918144,NaN,0.696896,0.658087,NaN,1.000000,0.886120
revenue7,0.686372,0.953494,NaN,0.767946,0.371575,NaN,0.886120,1.000000


In [16]:
answer_2_3 = """
3) Уместно ли допущение, что конверсия из кликов в установки зависит от платформы,
но не от страны? Выглядит не очень, но на этих данных не кажется, что я могу сделать лучше.

>Допустимо делать любые допущения, если это позволяет увеличить точность.
В целом, предложенное допущение вполне рабочее.
"""

In [17]:
# Если предположить, что клики конвертируются в установки (Google, FB, а Organic поместим в ошибку, 
# так как мы точно не знаем, сколько оттуда кликов), то можно составить регрессию:
model = ols("installs ~ clicks_g + clicks_f", data=totals).fit()
print(model.params)
model.summary()

Intercept    324.780354
clicks_g       0.377851
clicks_f       0.058831
dtype: float64


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:               installs   R-squared:                       0.727
Model:                            OLS   Adj. R-squared:                  0.695
Method:                 Least Squares   F-statistic:                     22.65
Date:                Sun, 11 Aug 2024   Prob (F-statistic):           1.61e-05
Time:                        23:10:31   Log-Likelihood:                -156.56
No. Observations:                  20   AIC:                             319.1
Df Residuals:                      17   BIC:                             322.1
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept    324.7804    175.266      1.853      0.081     -44.999     694.560
clicks_g       0.3779      0.088      4.279      0.001       0.192       0.564
clicks_f       0.0588      0.015      3.878      0.001       0.027       0.091
==============================================================================
Omnibus:                        3.295   Durbin-Watson:                   1.567
Prob(Omnibus):                  0.193   Jarque-Bera (JB):                1.846
Skew:                           0.731   Prob(JB):                        0.397
Kurtosis:                       3.280   Cond. No.                     1.39e+04
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.39e+04. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [18]:
# Коэффициенты статистически значимы, так что грех ими не воспользоваться.
coeff_f = model.params["clicks_f"]
coeff_g = model.params["clicks_g"]

# Округлять, конечно, не очень хорошо, но количество установок целочисленное (обычно).
totals["installs_g"] = (totals["clicks_g"] * coeff_g).apply(round)
totals["installs_f"] = (totals["clicks_f"] * coeff_f).apply(round)

totals["installs_o"] = totals["installs"] - (totals["installs_g"] + totals["installs_f"])

totals

,country,spend_g,spend_f,spend_o,clicks_g,clicks_f,clicks_o,installs,revenue7,installs_g,installs_f,installs_o
0,US,17443.027248,16009.235678,0.0,6435,22297,NaN,5063,11345.630884,2431,1312,1320
1,UK,0.000000,2151.368265,0.0,0,6014,NaN,2077,934.957527,0,354,1723
2,BR,0.000000,1865.899963,0.0,0,36982,NaN,1826,174.960320,0,2176,-350
3,DE,0.000000,1620.480017,0.0,0,4917,NaN,1784,3301.434703,0,289,1495
4,FR,3076.126498,2622.658306,0.0,1151,13124,NaN,1576,1013.666092,435,772,369
5,IT,1947.870521,2343.566877,0.0,850,20595,NaN,1474,678.665733,321,1212,-59
6,JP,17763.000442,0.000000,0.0,5099,0,NaN,1080,1177.084200,1927,0,-847
7,ES,0.000000,1599.437345,0.0,0,14406,NaN,894,439.404414,0,848,46
8,CA,0.000000,0.000000,0.0,0,0,NaN,801,826.125587,0,0,801
9,KR,0.000000,0.000000,0.0,0,0,NaN,602,377.700856,0,0,602


Регрессия провалилась, всё очень плохо: есть строки, в которых количество установок даже одного типа больше, чем суммарное количество. 

Попробуем вытащить конверсию из распределения установок:

In [19]:
total_installs.sort_index()

Facebook     3137
Google       3841
Organic     11851
dtype: int64

In [20]:
total_clicks = (totals
    .loc[:, ["clicks_f", "clicks_g", "clicks_o"]]
    .sum()
    .rename({
        "clicks_f": "Facebook",
        "clicks_g": "Google",
        "clicks_o": "Organic"
    }).sort_index()
)
total_clicks

Facebook    118568.0
Google       14180.0
Organic          0.0
dtype: float64

In [21]:
total_clicks_installs = pd.concat(
    [total_clicks, total_installs.sort_index()],
    axis=1
)

total_clicks_installs.columns = ["clicks", "installs"]
total_clicks_installs

,clicks,installs
Facebook,118568.0,3137
Google,14180.0,3841
Organic,0.0,11851


In [22]:
total_clicks_installs["ctr"] = total_clicks_installs["installs"] / total_clicks_installs["clicks"]
total_clicks_installs

,clicks,installs,ctr
Facebook,118568.0,3137,0.026457
Google,14180.0,3841,0.270874
Organic,0.0,11851,inf


In [23]:
coeff_f = total_clicks_installs.loc["Facebook", "ctr"]
coeff_g = total_clicks_installs.loc["Google", "ctr"]

totals["installs_g"] = (totals["clicks_g"] * coeff_g).apply(round)
totals["installs_f"] = (totals["clicks_f"] * coeff_f).apply(round)

totals

,country,spend_g,spend_f,spend_o,clicks_g,clicks_f,clicks_o,installs,revenue7,installs_g,installs_f,installs_o
0,US,17443.027248,16009.235678,0.0,6435,22297,NaN,5063,11345.630884,1743,590,1320
1,UK,0.000000,2151.368265,0.0,0,6014,NaN,2077,934.957527,0,159,1723
2,BR,0.000000,1865.899963,0.0,0,36982,NaN,1826,174.960320,0,978,-350
3,DE,0.000000,1620.480017,0.0,0,4917,NaN,1784,3301.434703,0,130,1495
4,FR,3076.126498,2622.658306,0.0,1151,13124,NaN,1576,1013.666092,312,347,369
5,IT,1947.870521,2343.566877,0.0,850,20595,NaN,1474,678.665733,230,545,-59
6,JP,17763.000442,0.000000,0.0,5099,0,NaN,1080,1177.084200,1381,0,-847
7,ES,0.000000,1599.437345,0.0,0,14406,NaN,894,439.404414,0,381,46
8,CA,0.000000,0.000000,0.0,0,0,NaN,801,826.125587,0,0,801
9,KR,0.000000,0.000000,0.0,0,0,NaN,602,377.700856,0,0,602


Всё ещё не очень, но лучше.

Можно ещё немножко исправить (хоть и костыльно): выставить верхним пределом для установок каждого типа известное суммарное количество.

In [24]:
totals["installs_g"] = totals.loc[:, ["installs", "installs_g"]].min(axis=1)
totals["installs_f"] = totals.loc[:, ["installs", "installs_f"]].min(axis=1)

totals["installs_o"] = totals["installs"] - (totals["installs_g"] + totals["installs_f"])
totals

,country,spend_g,spend_f,spend_o,clicks_g,clicks_f,clicks_o,installs,revenue7,installs_g,installs_f,installs_o
0,US,17443.027248,16009.235678,0.0,6435,22297,NaN,5063,11345.630884,1743,590,2730
1,UK,0.000000,2151.368265,0.0,0,6014,NaN,2077,934.957527,0,159,1918
2,BR,0.000000,1865.899963,0.0,0,36982,NaN,1826,174.960320,0,978,848
3,DE,0.000000,1620.480017,0.0,0,4917,NaN,1784,3301.434703,0,130,1654
4,FR,3076.126498,2622.658306,0.0,1151,13124,NaN,1576,1013.666092,312,347,917
5,IT,1947.870521,2343.566877,0.0,850,20595,NaN,1474,678.665733,230,545,699
6,JP,17763.000442,0.000000,0.0,5099,0,NaN,1080,1177.084200,1080,0,0
7,ES,0.000000,1599.437345,0.0,0,14406,NaN,894,439.404414,0,381,513
8,CA,0.000000,0.000000,0.0,0,0,NaN,801,826.125587,0,0,801
9,KR,0.000000,0.000000,0.0,0,0,NaN,602,377.700856,0,0,602


In [25]:
print("""{}\n\nSum: {}""".format(total_installs, total_installs.sum()))

Facebook     3137
Google       3841
Organic     11851
dtype: int64

Sum: 18829


In [26]:
calc_result = totals.loc[:, ["installs_g", "installs_f", "installs_o"]].sum().sort_index()
print("""{}\n\nSum: {}""".format(calc_result, calc_result.sum()))

installs_f     3135
installs_g     3540
installs_o    12154
dtype: int64

Sum: 18829


Хуже абсолютной истины, но не слишком.

In [27]:
totals[["revenue7", "spend_g", "spend_f", "clicks_g", "clicks_f", "installs", "installs_f", "installs_g", "installs_o"]].corr()

,revenue7,spend_g,spend_f,clicks_g,clicks_f,installs,installs_f,installs_g,installs_o
revenue7,1.000000,0.686372,0.953494,0.767946,0.371575,0.886120,0.371807,0.828788,0.793993
spend_g,0.686372,1.000000,0.634803,0.990243,0.203855,0.621872,0.204068,0.966427,0.346548
spend_f,0.953494,0.634803,1.000000,0.731347,0.549994,0.918144,0.550189,0.804341,0.794439
clicks_g,0.767946,0.990243,0.731347,1.000000,0.264338,0.696896,0.264563,0.992397,0.429655
clicks_f,0.371575,0.203855,0.549994,0.264338,1.000000,0.658087,1.000000,0.306048,0.508290
installs,0.886120,0.621872,0.918144,0.696896,0.658087,1.000000,0.658292,0.748878,0.920004
installs_f,0.371807,0.204068,0.550189,0.264563,1.000000,0.658292,1.000000,0.306281,0.508480
installs_g,0.828788,0.966427,0.804341,0.992397,0.306048,0.748878,0.306281,1.000000,0.493568
installs_o,0.793993,0.346548,0.794439,0.429655,0.508290,0.920004,0.508480,0.493568,1.000000


Пытаясь избежать мультиколлинеарности и максимизируя Adj.R^2, подбираем регрессию, которая должна объяснить выручку, исходя из имеющихся данных о Google, Facebook, Organic.

Без Intercept'а, потому что при нулевых показателях из воздуха она браться не должна.

In [28]:
model2 = ols("""
    revenue7 ~
    installs_g +
    spend_f +
    installs_o + 0
""", data=totals).fit()
print(model2.params)
model2.summary()

installs_g    1.302939
spend_f       0.451168
installs_o    0.508044
dtype: float64


<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:               revenue7   R-squared (uncentered):                   0.942
Model:                            OLS   Adj. R-squared (uncentered):              0.932
Method:                 Least Squares   F-statistic:                              92.52
Date:                Sun, 11 Aug 2024   Prob (F-statistic):                    9.88e-11
Time:                        23:10:31   Log-Likelihood:                         -157.84
No. Observations:                  20   AIC:                                      321.7
Df Residuals:                      17   BIC:                                      324.7
Df Model:                           3                                                  
Covariance Type:            nonrobust                                                  
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
installs_g     1.3029      0.622      2.095      0.051      -0.009       2.615
spend_f        0.4512      0.105      4.286      0.001       0.229       0.673
installs_o     0.5080      0.289      1.758      0.097      -0.102       1.118
==============================================================================
Omnibus:                        3.889   Durbin-Watson:                   2.399
Prob(Omnibus):                  0.143   Jarque-Bera (JB):                1.908
Skew:                           0.459   Prob(JB):                        0.385
Kurtosis:                       4.203   Cond. No.                         15.5
==============================================================================

Notes:
[1] R² is computed without centering (uncentered) since the model does not contain a constant.
[2] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [29]:
# Коэффициенты незначимы (кроме Facebook), но это лучшее, что у нас есть, так что:
inst_g_coeff = model2.params["installs_g"]
spend_f_coeff = model2.params["spend_f"]
inst_o_coeff = model2.params["installs_o"]

In [30]:
totals["revenue7_g"] = totals["installs_g"] * inst_g_coeff
totals["revenue7_f"] = totals["spend_f"] * spend_f_coeff
totals["revenue7_o"] = totals["installs_o"] * inst_o_coeff

# Наученный горьким опытом:
totals["revenue7_g"] = totals.loc[:, ["revenue7", "revenue7_g"]].min(axis=1)
totals["revenue7_f"] = totals.loc[:, ["revenue7", "revenue7_f"]].min(axis=1)
totals["revenue7_o"] = totals.loc[:, ["revenue7", "revenue7_o"]].min(axis=1)
# totals["revenue7_o2"] = totals["revenue7"] - (totals["revenue7_g"] + totals["revenue7_f"])

totals

,country,spend_g,spend_f,spend_o,clicks_g,clicks_f,clicks_o,installs,revenue7,installs_g,installs_f,installs_o,revenue7_g,revenue7_f,revenue7_o
0,US,17443.027248,16009.235678,0.0,6435,22297,NaN,5063,11345.630884,1743,590,2730,2271.023331,7222.853982,1386.959360
1,UK,0.000000,2151.368265,0.0,0,6014,NaN,2077,934.957527,0,159,1918,0.000000,934.957527,934.957527
2,BR,0.000000,1865.899963,0.0,0,36982,NaN,1826,174.960320,0,978,848,0.000000,174.960320,174.960320
3,DE,0.000000,1620.480017,0.0,0,4917,NaN,1784,3301.434703,0,130,1654,0.000000,731.108641,840.304315
4,FR,3076.126498,2622.658306,0.0,1151,13124,NaN,1576,1013.666092,312,347,917,406.517085,1013.666092,465.876093
5,IT,1947.870521,2343.566877,0.0,850,20595,NaN,1474,678.665733,230,545,699,299.676056,678.665733,355.122561
6,JP,17763.000442,0.000000,0.0,5099,0,NaN,1080,1177.084200,1080,0,0,1177.084200,0.000000,0.000000
7,ES,0.000000,1599.437345,0.0,0,14406,NaN,894,439.404414,0,381,513,0.000000,439.404414,260.626429
8,CA,0.000000,0.000000,0.0,0,0,NaN,801,826.125587,0,0,801,0.000000,0.000000,406.943021
9,KR,0.000000,0.000000,0.0,0,0,NaN,602,377.700856,0,0,602,0.000000,0.000000,305.842320


In [31]:
total_revenue_calc = totals.loc[:, ["revenue7_g", "revenue7_f", "revenue7_o"]].sum().sum()
total_revenue_true = totals["revenue7"].sum()

print("""
Рассчитанное: ${:,.2f}
Реальное:     ${:,.2f}
Разница:         {:,.2f} ({:.2%})

Приемлемо.
""".format(
        total_revenue_calc,
        total_revenue_true,
        total_revenue_calc - total_revenue_true,
        (total_revenue_calc - total_revenue_true) / total_revenue_true
    )
)


Рассчитанное: $21,419.26
Реальное:     $22,110.25
Разница:         -690.98 (-3.13%)

Приемлемо.



In [32]:
# А теперь приведём ответ к требуемому формату.
answer = totals.loc[:, ["country", "revenue7_g", "revenue7_f"]]

answer_g = (answer
    .loc[answer["revenue7_g"] > 0, ["country", "revenue7_g"]]
    .rename({"revenue7_g": "revenue7"}, axis=1)
    
)
answer_g["type"] = "Google"

answer_f = (answer
    .loc[answer["revenue7_f"] > 0, ["country", "revenue7_f"]]
    .rename({"revenue7_f": "revenue7"}, axis=1)
)
answer_f["type"] = "FB"

answer = (pd.concat(
        [answer_g, answer_f],
        axis=0
    )
    .sort_values(["type", "country"])
    .reset_index()
    .loc[:, ["type", "country", "revenue7"]]
)
answer

,type,country,revenue7
0,FB,BR,174.960320
1,FB,DE,731.108641
2,FB,ES,439.404414
3,FB,FR,1013.666092
4,FB,IT,678.665733
5,FB,UK,934.957527
6,FB,US,7222.853982
7,Google,FR,406.517085
8,Google,IT,299.676056
9,Google,JP,1177.084200


Чего-то не хватает.

Некоторые страны из исходного списка отсутствуют, потому что их `revenue7` оказался 0.

Исправим `left join`'ом:

In [33]:
full_answer = (by_type
    .loc[:, ["type", "country"]]
    .merge(
        answer,
        on=["type", "country"],
        how="left"
    )
    .fillna(0.0)
)

full_answer

,type,country,revenue7
0,FB,US,7222.853982
1,FB,FR,1013.666092
2,FB,IT,678.665733
3,FB,UK,934.957527
4,FB,BR,174.960320
5,FB,DE,731.108641
6,FB,ES,439.404414
7,FB,GP,0.000000
8,FB,FJ,0.000000
9,FB,TL,0.000000


Вот и наш ответ.

За исключением приемлемого расхождения исходного и рассчитанного `revenue7` и 2 костылей:

а) допущения, что конверсия из кликов в установки зависит от платформы, но не от страны, и округления расчётов, построенных на этой конверсии;

б) использования коэффициентов, статистически незначимых на уровне **0.05**, для расчётов распределения `revenue7` по типам и округления результатов этих расчётов

выглядит неплохо.

(Можно было бы ещё совсем бездушно нормализовать результат, вытащив процентное соотношение и умножив его на известную истинную сумму, но не очень хочется добавлять третий костыль.)